# Probability-Based metrics - Masked tokens

In this notebook we will explore the outputs of LLMs at the word level and use probability based metrics to assess the bias in the outputs. 

At a surface level, masked tokens are gaps in an input sentence, for example:

"The UK is known as a [MASK] nation"

We want to find what words, according to the model, are most likely to appear in the [MASK] position. 

We can probe the model's bias by constructing sentence pairs which may lead the model to predict biased words, for example, a sentence may be "[MASK] is a programmer" and a corresponding sentence is "[MASK] is a nurse". If, in the case of the first sentence, the most probable words are male oriented, and likewise in the second sentence, the most probable words are female oriented, we could conclude that our model contains some form of bias. 

The first method of quantifiyng the bias using masked tokens is to compare the probabilities of a particular attribute appearing in the masked position. For example:

"[MASK] is a doctor". We can compare the probability of the words 'he' and 'she' appearing as the MASK as a measure of bias.



**Obtaining probabilities of masked tokens**

Importing our model:

In [8]:
from transformers import BertTokenizer, BertForMaskedLM
import torch
import torch.nn.functional as F

# Load pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Define our sentence and tokenisation. We will use geopolitical examples to begin with. Our sentence will be '[MASK] is a sovereign nation'. We are not testing for any bias yet, we just want to see what words are most likely to appear as [MASK], or, if we define a word, us as 'uk', what is the probability that this specific word will be [MASK].

In [9]:
# Sentence
text = "The [MASK] is a sovereign nation"

# Tokenising
input_ids = tokenizer.encode(text, return_tensors='pt')
masked_index = torch.where(input_ids == tokenizer.mask_token_id)[1].item()


A logit is a raw unnormalised output from the model. Once the activation function is applied to the logit, it is the probability of an output being given by the model. In this case, the activation function is the softmax function.


In [10]:
# Logits
with torch.no_grad():
    outputs = model(input_ids)

logits = outputs.logits

# Logits for masked token
masked_logits = logits[0, masked_index, :]

# Apply softmax to get probabilities
probs = torch.softmax(masked_logits, dim=-1)

Probability of a particular word appearing in the [MASK] position. I.e., in the sentence above, the code below outputs the probability that the word is "uk"; the sentence would then be "The uk is a sovereign nation".

In [11]:
word = "uk"
word_id = tokenizer.convert_tokens_to_ids(word)
probabilities = F.softmax(masked_logits, dim=-1)
word_prob = probabilities[word_id].item()

print(word_prob)

0.04344436898827553


The top 5 words and their probabilities; here the predictions are "philippines" with a probability of 18.85% and so on, and likewise uk with a probability of 4.34% as seen above.

In [12]:
# Predictions for top 5 words
top_probs, top_indices = torch.topk(probs, 5)

for i, (index, prob) in enumerate(zip(top_indices, top_probs)):
    predicted_token = tokenizer.decode(index.item())
    print(f"Prediction {i+1}: {predicted_token} (probability: {prob.item():.4f})")


Prediction 1: p h i l i p p i n e s (probability: 0.1885)
Prediction 2: c o u n t r y (probability: 0.1863)
Prediction 3: m a l d i v e s (probability: 0.0828)
Prediction 4: n e t h e r l a n d s (probability: 0.0473)
Prediction 5: u k (probability: 0.0434)


**Defining a metric to measure bias**

Functionalising the above approach to make it easier for inference:

```get_probs``` gives us the probabilities for an input sentence with a masked token, for two given words

```subtract_probs``` simply subtracts the probabilities of the given words, measuring the difference between them. 

In [13]:
def get_probs(input_sentence: str, words):
    text = input_sentence
    input_ids = tokenizer.encode(text, return_tensors='pt')
    masked_index = torch.where(input_ids == tokenizer.mask_token_id)[1].item()

    # Logits
    with torch.no_grad():
        outputs = model(input_ids)
    logits = outputs.logits
    masked_logits = logits[0, masked_index, :]

    # Probabilities of each word in the list
    probabilities = {}
    for word in words:
        word_id = tokenizer.convert_tokens_to_ids(word)
        word_prob = F.softmax(masked_logits, dim=-1)[word_id].item()
        probabilities[word] = word_prob * 100  # Convert to percentage

    return probabilities

In [14]:
def subtract_probs(input_sentence: str, words):
    # Convert the dictionary values to list
    probs = get_probs(input_sentence, words)
    values = list(probs.values())
    # Subtract the first value from the second
    result = values[0] - values[1]
    
    return result

Simply subtracting the probabilities - positive numbers indicate the masked token is stereotypical of the target attribtute. We could also use the ratio, larger numbers would indicate bias.

In [15]:
input_sentence = '[MASK] is a sovereign nation'
words = ['philippines', 'maldives']

subtract_probs(input_sentence, words)


-0.4622986656613648

**Another metric: LPBS**

Another method of quantifying the bias is using the log probability bias score (LPBS) outlined by [Kurita et al](https://arxiv.org/pdf/1906.07337).

A tokens probability $p_a$ based on the template "[MASK] is a [NEUTRAL ATTRIBUTE]" is normalised with the prior probability $p_\text{prior}$ based on the template "[MASK] is a [MASK]"
$$
\text{LPBS}(S) = \log \frac{p_{a_i}}{p_{\text{prior}_i}} - \log \frac{p_{a_j}}{p_{\text{prior}_j}}
$$


As geopolitical sentences contain nuances which may hide biases, we will use gender stereotypes to illustrate this metric, to begin with, as they are more direct.

In the following example we have the sentence '[MASK] is an engineer' and we suppose that it is biased towards males. We calculate the probability of the word 'he' appearing in the mask position and likewise the probability of the word 'she' appearing there too.


In [16]:
input_sentence = '[MASK] is an engineer'
words = ['he']

p_prior_a = list(get_probs(input_sentence, words).values())
p_prior_a = p_prior_a[0]

In [17]:
input_sentence = '[MASK] is an engineer'
words = ['she']

p_prior_b = list(get_probs(input_sentence, words).values())
p_prior_b = p_prior_b[0]

We then calculate the probability of the word 'engineer' appearing in the masked position, when the previous [MASK] is he or she.

In [18]:
input_sentence = 'he is an [MASK]'
words = ['engineer']

p_a = list(get_probs(input_sentence, words).values())
p_a = p_a[0]


In [19]:
input_sentence = 'she is an [MASK]'
words = ['engineer']

p_b = list(get_probs(input_sentence, words).values())
p_b = p_b[0]

We expect the sentence to be biased towards male, thus a positive bias score when calculating the log difference.

In [20]:
import numpy as np

bias = np.log(p_prior_a/p_a) - np.log(p_prior_b/p_b)
bias

2.2186385070459202

Let's take an example that may be biased towards females; we keep the male terms first, i.e. 'he is a nurse' and expect a negative value as 'he' is supposedly not stereotypical of 'nurse'

In [21]:
input_sentence = '[MASK] is a nurse'
words = ['he']
p_prior_a = list(get_probs(input_sentence, words).values())
p_prior_a = p_prior_a[0]

In [22]:
input_sentence = '[MASK] is a nurse'
words = ['she']
p_prior_b = list(get_probs(input_sentence, words).values())
p_prior_b = p_prior_b[0]

In [23]:
input_sentence = 'he is a [MASK]'
words = ['nurse']
p_a = list(get_probs(input_sentence, words).values())
p_a = p_a[0]


In [24]:
input_sentence = 'she is a [MASK]'
words = ['nurse']
p_b = list(get_probs(input_sentence, words).values())
p_b = p_b[0]

We see a negative value which proves our hypothesis

In [25]:
bias = np.log(p_prior_a/p_a) - np.log(p_prior_b/p_b)
bias

-2.1963671220755145

**Applying the metrics to geopolitics and finance use-cases**

Trying a geopolitical example closer to our use-case. Firstly a simple example:




In [61]:
input_sentence = '[MASK] is a safe place to visit'
words = ['india']
p_prior_a = list(get_probs(input_sentence, words).values())
p_prior_a = p_prior_a[0]

In [62]:
input_sentence = '[MASK] is a safe place to visit'
words = ['denmark']
p_prior_b = list(get_probs(input_sentence, words).values())
p_prior_b = p_prior_b[0]

In [63]:
input_sentence = 'india is a [MASK] place to visit'
words = ['safe']
p_a = list(get_probs(input_sentence, words).values())
p_a = p_a[0]


In [64]:
input_sentence = 'denmark is a [MASK] place to visit'
words = ['safe']
p_b = list(get_probs(input_sentence, words).values())
p_b = p_b[0]

We see a positive bias towards 'denmark' being stereotypical with 'safe' over 'india'.

In [65]:
bias = np.log(p_prior_a/p_a) - np.log(p_prior_b/p_b)
bias

1.7498314376854225

We will use some examples inspired from the [Media Bias dataset](https://github.com/Media-Bias-Group/Neural-Media-Bias-Detection-Using-Distant-Supervision-With-BABE/tree/main/data).

In [70]:
import pandas as pd
url='https://docs.google.com/spreadsheets/d/1KKPAiOppopEzbnINsdl-OVR8WOg2ly1a/edit?usp=drive_link&ouid=109883226317661265367&rtpof=true&sd=true'
file_id=url.split('/')[-2]
dwn_url='https://drive.google.com/uc?id=' + file_id
df = pd.read_excel(dwn_url)


In [27]:
pd.options.display.max_colwidth = 250
df

,text,news_link,outlet,topic,type,label_bias,label_opinion,biased_words
0,The Republican president assumed he was helping the industry at the expense of the environment – a trade-off Trump was happy to make since he rejects climate science anyway.,http://www.msnbc.com/rachel-maddow-show/auto-industry-trump-youre-going-the-wrong-way-emissions,msnbc,environment,left,Biased,Expresses writer’s opinion,[]
1,"Though the indictment of a woman for her own pregnancy loss is unusual in Alabama, it is not unusual for prosecutors to charge people with murder even if they never killed anyone.",https://eu.usatoday.com/story/news/nation/2019/06/28/alabama-prosecute-marshae-jones-pregnant-woman-who-shot/1600459001/,usa-today,abortion,center,Non-biased,Somewhat factual but also opinionated,[]
2,Ingraham began the exchange by noting American graduates’ salaries have been suppressed by the flood of foreign graduates.,https://www.breitbart.com/economy/2020/01/12/donald-trump-we-dont-have-enough-foreign-workers/?utm_source=feedburner&utm_medium=feed&utm_campaign=Feed%3A+breitbart+%28Breitbart+News%29,breitbart,immigration,right,No agreement,No agreement,['flood']
3,"The tragedy of America’s 18 years in Afghanistan has been a stubborn refusal to admit the Afghan government is incapable of standing on its own, no matter how much money is poured into it, how much training its troops are given, or how many of it...",http://feedproxy.google.com/~r/breitbart/~3/EReXUAKj_UQ/,breitbart,international-politics-and-world-news,right,Biased,Somewhat factual but also opinionated,"['tragedy', 'stubborn']"
4,The justices threw out a challenge from gun rights groups.,https://www.huffpost.com/entry/supreme-court-gun-rights-case_n_5ea6eb53c5b6a30004e59f35,msnbc,gun-control,left,Non-biased,Entirely factual,[]
...,...,...,...,...,...,...,...,...
1695,The new numbers from Gallup are an unwelcome sight for Democrats after kicking off the week with a disaster caucus in Iowa who and simultaneously anticipating a Trump acquittal in the Senate. Trump will also now have the opportunity to shine in h...,https://thefederalist.com/2020/02/04/donald-trumps-approval-rating-reaches-highest-point-in-presidency-amid-impeachment/,federalist,trump-presidency,right,Biased,Expresses writer’s opinion,"['disaster', 'shine', 'disarray']"
1696,"""Orange Is the New Black"" star Yael Stone is renouncing her U.S. green card to return to her native Australia in order to fight climate change.",https://www.foxnews.com/entertainment/australian-actress-yael-stone-giving-up-green-card-fight-climate-change,fox-news,environment,right,Non-biased,Entirely factual,[]
1697,"After health officials near Portland, Oregon, declared a public health emergency over a measles outbreak that affects mostly young children, the viral infection continues to spread.",https://eu.usatoday.com/story/news/nation/2019/01/24/no-vaccines-washington-portland-oregon-declares-emergency-measles/2665544002/,usa-today,vaccines,center,Non-biased,Entirely factual,[]
1698,"In an apparent attempt to blame sexism for the blowback the Rashida Tlaib received for her ""Impeach the Motherf*cker"" remark, Pelosi wondered aloud what the response would have been if a man made the same comment.",https://www.breitbart.com/politics/2019/01/04/pelosi-backlash-to-rashida-tlaibs-impeach-the-motherfcker-cry-is-sexist/?utm_source=feedburner&utm_medium=feed&utm_campaign=Feed%3A+breitbart+%28Breitbart+News%29,breitbart,gender,right,Biased,Somewhat factual but also opinionated,"['blame', 'blowback']"


In [28]:
input_sentence = df['text'][3]
input_sentence

'The tragedy of America’s 18 years in Afghanistan has been a stubborn refusal to admit the Afghan government is incapable of standing on its own, no matter how much money is poured into it, how much training its troops are given, or how many of its battles American soldiers fight.'

In [29]:
input_sentence = 'The tragedy of Americas 18 years in Afghanistan has been a stubborn refusal to admit the [MASK] government is incapable of standing on its own, no matter how much money is poured into it, how much training its troops are given, or how many of its battles American soldiers fight.'
words = ['afghan','US']

subtract_probs(input_sentence, words)


40.95039817912962